<a href="https://colab.research.google.com/github/ggruszczynski/gpu_colab/blob/main/80_gpu_aos_vs_soa_mem_layout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Memory layout - porting issues

Consider a set of points (x,y,z) describing some geometry...

![Mesh-of-a-F1-car.jpg](https://github.com/ggruszczynski/gpu_colab/blob/main/lectures/figures/Mesh-of-a-F1-car.jpg?raw=1)

In [ ]:
!nvidia-smi

Thu Jun 29 12:43:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from numba import cuda
from numba import jit
import numpy as np
import math
from numba import vectorize, int8, int32, int64, float32, float64
import matplotlib.pyplot as plt

np.set_printoptions(precision=1, suppress=True)

%matplotlib inline

M = 2**13     # ~1000MB, 2**13=8192 on GPU SOA is better than AOS
# M = 2**11       # ~100MB, 2**11=2048
# M = 2**10     # ~10MB, 2**10=1024 on both CPU & GPU: SOA ~ AOS
# M = 2**6      # <1MB, 2**6=64 no difference between AOS and SOA, CPU is faster



N = 3*M**2

a = np.arange(N, dtype=np.float64) # [0...N] on the host
print(f"Total number of elements: {N} \nDimensions of the matrix MxM: {M}x{M} \nMemory size of array in [MB]: {a.nbytes/1E6}")


Total number of elements: 201326592 
Dimensions of the matrix MxM: 8192x8192 
Memory size of array in [MB]: 1610.612736


In [ ]:
# threadsperblock = (8,8)  # for M = 2**6 = 64
threadsperblock = (32,32)
blockspergrid_x = math.ceil(M / threadsperblock[0])
blockspergrid_y = math.ceil(M / threadsperblock[1])
# blockspergrid_z = math.ceil(d_a.shape[2] / threadsperblock[2])
blockspergrid = (blockspergrid_x, blockspergrid_y)
total_threads_in_block = threadsperblock[0]*threadsperblock[1]
assert total_threads_in_block <= 1024 # hardware limit
assert total_threads_in_block <= M # avoid useless threads

print(f"Blockspergrid: {blockspergrid}, Total number of threads in a block: {total_threads_in_block}")

Blockspergrid: (256, 256), Total number of threads in a block: 1024


# AOS (Array of Structures)

The MxM matrix stores Points(x,y,z) in its most inner dimension


In [ ]:
arr_aos = a.reshape(M,M,3)        # The MxM matrix stores Points(x,y,z) in its most inner dimension
aos_out = np.zeros_like(arr_aos)
print(f" Shape: {arr_aos.shape} \n Strides {arr_aos.strides} \n Content {arr_aos[0,0:3,:]}")

d_arr_aos = cuda.to_device(arr_aos)           # Copy array from host to the device
d_aos_out = cuda.device_array_like(d_arr_aos) # preallocate an arracy filled with 0

# print(f"GPU result:\n {d_aos_out.copy_to_host()[0,0:3,:]}")

 Shape: (8192, 8192, 3) 
 Strides (196608, 24, 8) 
 Content [[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]


In [ ]:
@jit(nopython=True)
def cpu_aos(M, A_in,  A_out):
    for tidy in range(M):
      for tidx in range(M):
        # A_out[tidy][tidx][0] = A_in[tidy][tidx][0]
        # A_out[tidy][tidx][1] = A_in[tidy][tidx][1]
        A_out[tidy][tidx][2] = A_in[tidy][tidx][2] + 1000

@cuda.jit
def kernel_gpu_aos(M, A_in, A_out):
    tidx, tidy = cuda.grid(2)
    # The above is equivalent to the following 2 lines of code:
    # x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    # y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    # loop over all points in domain (except boundary)
    if (tidx < M and tidy < M):
        # A_out[tidx][tidy][0] = A_in[tidx][tidy][0]
        # A_out[tidx][tidy][1] = A_in[tidx][tidy][1]
        A_out[tidx][tidy][2] = A_in[tidx][tidy][2] + 1000

@cuda.jit
def kernel_gpu_aos2(M, A_in, A_out):
    tidx, tidy = cuda.grid(2)
    # The above is equivalent to the following 2 lines of code:
    # x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    # y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    # loop over all points in domain (except boundary)
    if (tidx < M and tidy < M):
        # A_out[tidy][tidx][0] = A_in[tidy][tidx][0]
        # A_out[tidy][tidx][1] = A_in[tidy][tidx][1]
        A_out[tidy][tidx][2] = A_in[tidy][tidx][2] + 1001

In [ ]:
%timeit -r 7 -n 5000 kernel_gpu_aos[blockspergrid, threadsperblock](M, d_arr_aos, d_aos_out)

25 ms ± 1.82 ms per loop (mean ± std. dev. of 7 runs, 5000 loops each)


In [ ]:
%timeit -r 7 -n 5000 kernel_gpu_aos2[blockspergrid, threadsperblock](M, d_arr_aos, d_aos_out)

20.3 ms ± 398 µs per loop (mean ± std. dev. of 7 runs, 5000 loops each)


In [ ]:
# %timeit -r 7 -n 1000 cpu_aos(M, arr_aos, aos_out) #to slow

In [ ]:
# print(f"CPU result:\n {aos_out[0,0:3,:]}")
print(f"GPU result:\n {d_aos_out.copy_to_host()[0,0:3,:]}")

GPU result:
 [[   0.    0. 1003.]
 [   0.    0. 1006.]
 [   0.    0. 1009.]]


## Cleanup

In [ ]:
del arr_aos
del aos_out

del d_arr_aos
del d_aos_out

# SOA (Structure of Arrays)

Each of the MxM matrices stores only the x, y or z coordinates.

In [ ]:
arr_soa = a.reshape(3,M,M)                     # Each of the most outer dimensions stores a MxM matrix stores only x, y or z coordinates.
out_soa = np.zeros_like(arr_soa)

print(f" Shape: {arr_soa.shape} \n Strides {arr_soa.strides} \n Content {arr_soa[2,0,:10]}")

d_arr_soa = cuda.to_device(arr_soa)            # Copy array from host to the device
d_soa_out = cuda.device_array_like(d_arr_soa)  # preallocate an arracy filled with 0

 Shape: (3, 8192, 8192) 
 Strides (536870912, 65536, 8) 
 Content [1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08
 1.3e+08]


In [ ]:
@jit(nopython=True)
def cpu_soa(M, A_in,  A_out):
    for tidy in range(M):
      for tidx in range(M):
        # A_out[0][tidy][tidx] = A_in[0][tidy][tidx]
        # A_out[1][tidy][tidx] = A_in[1][tidy][tidx]
        A_out[2][tidy][tidx] = A_in[2][tidy][tidx] + 1000

@cuda.jit
def kernel_gpu_soa(M, A_in,  A_out):
    tidx, tidy = cuda.grid(2)
    # The above is equivalent to the following 2 lines of code:
    # x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    # y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    # loop over all points in domain (except boundary)
    if (tidx < M and tidy < M):
      #  A_out[0][tidx][tidy] = A_in[0][tidx][tidy]
      #  A_out[1][tidx][tidy] = A_in[1][tidx][tidy]
       A_out[2][tidx][tidy] = A_in[2][tidx][tidy] + 1000

@cuda.jit
def kernel_gpu_soa2(M, A_in,  A_out):
    tidx, tidy = cuda.grid(2)
    # The above is equivalent to the following 2 lines of code:
    # x = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    # y = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y

    # loop over all points in domain (except boundary)
    if (tidx < M and tidy < M):
      #  A_out[0][tidy][tidx] = A_in[0][tidy][tidx]
      #  A_out[1][tidy][tidx] = A_in[1][tidy][tidx]
       A_out[2][tidy][tidx] = A_in[2][tidy][tidx] + 1001

In [ ]:
%timeit -r 7 -n 5000 kernel_gpu_soa[blockspergrid, threadsperblock](M, d_arr_soa, d_soa_out)

9.59 ms ± 711 µs per loop (mean ± std. dev. of 7 runs, 5000 loops each)


In [ ]:
%timeit -r 7 -n 5000 kernel_gpu_soa2[blockspergrid, threadsperblock](M, d_arr_soa, d_soa_out)

4.6 ms ± 387 µs per loop (mean ± std. dev. of 7 runs, 5000 loops each)


In [ ]:
# %timeit -r 7 -n 1000 loops_no cpu_soa(M, arr_soa, out_soa)  #to slow

In [ ]:
# print(f"CPU result:\n {out_soa[2,0,:10]}")
print(f"GPU result:\n {d_soa_out.copy_to_host()[2,0,:10]}")

GPU result:
 [1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08 1.3e+08
 1.3e+08]


## Cleanup

In [ ]:
del arr_soa
del out_soa

del d_arr_soa
del d_soa_out

## timeit: General advise regarding the values of repeat and number

A computer has different "clocks" to measure times. These clocks have different "ticks" (depending on the OS). For example it could measure seconds, milliseconds or nanoseconds - these ticks are called the granularity of the clock.

If the duration of the execution is smaller or roughly equal to the granularity of the clock one cannot get representative timings.

The `number` (`-n`) and `repeat` (`-r`) are separate arguments because they serve different purposes. The number controls how many executions (aka loops) are done for each timing and it's used to get representative timings. The repeat argument controls how many timings are done and its use is to get accurate statistics. IPython uses the mean or average to calculate the run-time of the statement of all repetitions and then divides that number with number. So it measures the average of the averages.

If you want to modify either `number` or `repeat` then you should set number to the minimum value possible without running into the granularity of the timer. In my experience number should be set so that number executions of the function take at least 10 microseconds (0.00001 seconds) otherwise you might only "time" the minimum resolution of the "timer".

The repeat should be set as high as possible. Having more repeats will make it more likely that you really find the real best or average. However more repeats will take longer so there's a trade-off as well.

Source: <https://stackoverflow.com/questions/48258008/n-and-r-arguments-to-ipythons-timeit-magic>




### Exercise

Try to `timeit` the kernels without specifying the `-r` and `-n` flags explicitly.